In [3]:
# imports -----------------------------------------------------------------
import librosa
from PIL import Image
import numpy as np
import os

import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torchvision.transforms as T_vision

import torch.nn as nn
import torch.optim as optim


In [4]:
# organize directories ----------------------------------------------------
cwd = os.getcwd()
# stores train fricative spectrogram images for all sentences and speakers
spg_train_dir = cwd + '/spg_data/train/'

# stores val fricative spectrogram images for all sentences and speakers
spg_val_dir = cwd + '/spg_data/val/'

# create datasets and dataloaders -----------------------------------------
train_ds = datasets.ImageFolder(root=spg_train_dir, 
                                transform=transforms.ToTensor())
train_loader = DataLoader(train_ds, batch_size=64)

val_ds = datasets.ImageFolder(root=spg_val_dir, 
                                transform=transforms.ToTensor())
val_loader = DataLoader(val_ds, batch_size=64)

In [5]:
# CUDA --------------------------------------------------------------------
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('**setup note** using device: ', device)

**setup note** using device:  cuda


In [15]:
# CNN model ---------------------------------------------------------------
class TwoLayerCNN(nn.Module):
  def __init__(self, c1, c2, c3):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=c1, out_channels=c2, kernel_size=3, stride=1, padding=0)
    self.conv2 = nn.Conv2d(in_channels=c2, out_channels=c3, kernel_size=3, stride=1, padding=0)

    self.relu = nn.ReLU()

    self.dropout = nn.Dropout(0.5)

    self.maxPool1 = nn.MaxPool2d((4,3), stride=(1,3))
    self.maxPool2 = nn.MaxPool2d((1,3), stride=(1,3))

    self.fc1 = nn.Linear(in_features=8, out_features=2)
    self.fc2 = nn.Linear(in_features=2, out_features=2)

  def forward(self, x):
    #   1. conv ReLU
    x = self.relu(self.conv1(x))
    #   2. maxpooling, dropout 50%
    x = self.dropout(self.maxPool1(x))
    #   3. conv ReLU
    x = self.relu(self.conv2(x))
    #   4. maxpooling
    x = self.maxPool2(x)
    #   5. FC ReLU, dropout 50%
    x = self.dropout(self.relu(self.fc1(x)))
    #   6. FC ReLU, dropout 50%
    x = self.dropout(self.relu(self.fc2(x)))
    #   7. softmax
    # softmax is already included in pytorch entropy loss func, so no need
    # to add it here
    return x
  
def validate(model, device, val_loader):
  correct = 0
  total = 0
  model.eval()

  with torch.no_grad():
    for input, target in val_loader:
      input = input.to(device)
      target = target.to(device)

      output = model(input)
      _, prediction = output.max(axis=1)
      correct += (prediction == target).sum()
      total += prediction.size(0)

    accuracy = float(correct)/total
  
  return accuracy


def train(model, device, train_loader, val_loader, lr, m, wd, epochs,
           print = 100):
  loss_curve = []
  acc_curve = []

  model = model.to(device)
  model.train()
  optimizer = optim.SGD(model.parameters(), lr=lr, momentum=m,
                         weight_decay=wd, nesterov=True)

  for epoch in range(epochs):
    for iter, (input, target) in enumerate(train_loader):
      input = input.to(device)
      target = target.to(device)

      output = model(input)
      loss = nn.functional.cross_entropy(output, target)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_curve.append(loss.item())

      # TODO: can move to print block if too slow        
      accuracy = validate(model, device, val_loader)
      acc_curve.append(accuracy)


      if (iter+1) % print == 0:
        print(f'Running Epoch {epoch} and Iteration {iter}: Loss is {loss.item()}')

        print(f'Accuracy against validation set is {accuracy}')

  return loss_curve, acc_curve

In [16]:
layers = 1
channel1 = 3
channel2 = 1024
num_classes = 2
lr = 0.002
m = 0.9
wd = 0.001
epochs = 300

model = TwoLayerCNN(channel1, channel2, num_classes)
print(model)
loss_curve, acc_curve = train(model, device, train_loader, val_loader, lr, m, wd, epochs)

TwoLayerCNN(
  (conv1): Conv2d(3, 1024, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(1024, 2, kernel_size=(3, 3), stride=(1, 1))
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (maxPool1): MaxPool2d(kernel_size=(4, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  (maxPool2): MaxPool2d(kernel_size=(1, 3), stride=(1, 3), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=2, bias=True)
)


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of size: : [2]